# How are code changes affecting the size of the Deno binary?

[scarf](https://github.com/scarf005) on the Deno Discord had this question:

> Are there any guides to building a 'stripped-down' version of `deno` from source? for example, i'd like to build a deno binary that runs typescript files but other additional features like `check`, `fmt`, `jupyter`, `lsp` removed. this would be useful to reduce binary size in docker container

I figured the addition of Jupyter code was pretty small (main new requirement is zeromq, the rest is all protocols that Jupyter handles). To test that hypothesis, I calculated the Deno size between Linux canaries from dl.deno.land.

## Methodology

- Pull the last N commits via GitHub's GraphQL API
- Perform a `HEAD` against the canary linux asset URL to get the size
- If a commit download isn't found, skip it


In [43]:
import { fetchCanariesSinceTag, Canary } from "./mod.ts";

const canaries = await fetchCanariesSinceTag("v1.36.0");

const jupyterLines = canaries.filter((point: Canary) =>
  point.subject.includes("jupyter")
);


In [44]:
const style = `
.jupyter-orange {
  color: orange;
}

.container footer {
  margin-top: 32px;
  font-size: 0.75em;
}

.container footer a {
  color: black;
}
`;


In [45]:
import Plot from "https://deno.land/x/plot@0.0.2/mod.ts";

const el = Plot.plot({
  marginLeft: 100,
  height: 300,
  width: 800,
  marks: [
    Plot.lineY(canaries, {
      x: "authoredDate",
      y: "size",
    }),
    Plot.ruleY([0]),
    Plot.ruleX(jupyterLines, {
      stroke: "orange",
      x: "authoredDate",
      y2: "size",
    }),
    // Convert to MB formatting
    Plot.axisY({
      tickFormat: (d: number) =>
        `${(d / (1024 * 1024)).toLocaleString(undefined, {
          maximumFractionDigits: 2,
        })} MB`,
    }),
  ],
});

Deno.jupyter.html`
<div><style>${style}</style>
  <div class="container">
  <h2>Size of the Deno Canary on Linux over time (in commits)</h2>
  <h3>Commits that contain Jupyter related changes are in <span class="jupyter-orange">orange</span></h3>
  ${el.toString()}

  <footer>
    View <a href="https://github.com/rgbkrk/deno-canary-sizes">deno-canary-sizes</a> on GitHub.
  </footer>
  </div>`;


Size of the Deno Canary on Linux over time (in commits) 
 Commits that contain Jupyter related changes are in orange 
 6 Aug 13 20 27 3 Sep 10 17 24 1 Oct 8 15 authoredDate → <path d="M780,20.018L779.093,20.02L776.593,20.015L775.083,20.028L746.678,20.013L746.239,20.013L742.659,20.051L740.866,20.055L738.321,20.07L738.011,20.123L737.821,20.191L737.599,20.201L734.717,20.254L728.621,20.268L725.185,20.265L725.076,20.292L720.85,20.292L720.753,20.288L718.979,20.306L717.451,20.294L715.599,20.292L714.876,20.313L713.224,20.291L708.863,20.309L707.926,20.296L705.909,20.306L705.888,20.304L704.53,20.31L703.791,20.316L702.557,20.317L702.445,20.312L697.178,20.312L696.073,20.308L691.994,20.313L685.061,20.276L685.016,20.271L684.988,20.276L684.663,20.272L682.068,20.305L676.102,20.276L675.515,20.436L675.344,20.497L674.771,20.512L674.631,20L674.255,20.009L672.758,20.002L672.369,20.049L671.539,20.16L670.893,20.198L668.716,20.194L667.773,20.158L666.655,20.174L666.431,20.329L665.875,20.338L662.558,20.319L659.302,20.343L657.938,20.344L657.916,20.347L648.244,20.706L644.524,20.698L642.331,20.683L639.935,20.659L627.542,20.664L626.952,20.129L626.865,20.13L624.515,20.129L620.726,20.143L619.705,20.128L619.292,20.24L617.277,20.232L610.845,20.231L608.586,20.233L602.083,20.227L601.087,20.245L598.983,20.214L598.981,20.333L597.836,20.326L597.181,20.325L595.772,20.336L594.828,20.315L593.94,20.332L593.764,20.337L593.492,20.402L592.813,20.371L592.456,20.35L592.363,20.774L590.388,20.771L589.055,20.754L588.9,20.954L588.536,20.884L586.195,20.879L585.508,20.897L585.298,20.96L585.239,20.902L584.789,20.9L581.081,20.897L579.833,20.911L574.601,20.917L574.431,20.92L572.461,21.097L525.743,22.313L524.355,22.323L523.645,22.315L519.726,22.386L519.008,22.363L518.451,22.397L518.209,22.453L518.179,22.456L518.117,22.426L517.776,22.441L517.349,22.464L516.904,22.441L516.852,22.458L516.52,22.481L516.181,22.466L507.662,22.482L507.457,22.49L507.379,22.481L503.888,22.453L502.04,22.452L500.45,22.454L500.198,22.494L497.737,22.492L494.253,22.472L492.288,22.504L490.423,23.914L488.854,24.954L488.513,24.943L488.222,24.953L482.119,24.929L481.659,24.954L480.969,25.487L480.858,25.426L480.528,25.46L480.416,25.53L479.842,25.49L479.177,25.489L476.043,25.487L475.512,25.964L474.931,25.953L472.835,25.924L472.783,25.924L472.729,26.894L470.659,26.906L470.427,26.913L461.054,27.224L460.324,27.238L460.308,27.775L459.396,27.776L459.272,27.826L459.187,27.818L455.427,27.846L455.111,27.869L454.764,28.006L454.367,27.996L453.008,28.161L451.179,28.143L443.969,28.15L441.257,28.163L434.69,28.149L434.521,28.448L434.305,28.419L432.772,28.416L425.271,28.399L424.758,28.467L424.525,28.48L424.202,28.462L424.023,28.468L423.537,28.473L422.66,28.482L419.54,28.481L418.058,28.544L416.844,28.515L416.318,28.544L415.509,29.33L415.399,29.338L413.944,29.424L411.621,29.898L405.851,29.883L404.611,29.897L404.375,29.895L401.772,29.933L401.234,30.001L396.414,29.998L394.528,30.027L389.399,30.02L389.284,30.002L385.231,30.011L381.476,29.99L373.723,29.968L368.699,29.94L365.314,29.961L361.239,29.978L360.442,30.001L359.882,30.024L358.084,30.041L357.058,30.019L351.894,30.042L351.212,29.983L349.2,29.997L300.057,33.797L298.377,33.793L298.358,33.788L297.772,33.794L297.137,33.844L297.085,34.566L295.597,34.551L295.25,34.578L294.075,34.577L292.337,34.559L286.441,34.57L285.398,34.565L285.391,34.583L285.247,34.544L283.16,34.567L279.193,34.75L273.215,24.987L269.657,24.991L266.761,24.995L266.251,24.992L265.038,24.984L263.367,24.972L255.655,25.467L254.966,25.58L252.458,25.595L248.663,25.607L246.475,25.583L246.271,25.602L237.555,25.589L235.078,25.58L229.918,25.596L229.04,25.599L228.677,25.589L227.817,25.579L223.245,25.686L221.146,25.666L220.83,25.688L220.581,25.715L220.535,25.72L220.237,25.716L219.091,25.925L218.202,26.035L210.292,26.035L209.88,26.047L209.307,26.034L205.085,26.027L203.969,26.058L202.224,25.933L202.192,25.934L199.25,25.935L199.185,25.954L196.493,25.942L194.586,25.949L192.277,25.961L178.744,25.944L176.052,25.937L175.823,35.632L175.139,35.63